# YOLOv8 Ensemble

## Libraries

In [1]:
from ultralytics import YOLO
import torch
import torchvision

## Data Setup

In [ ]:
print(torch.cuda.is_available())
print(torchvision.__version__)

## Hyperparameter Tuning

In [ ]:
model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')

model.tune(data='crystals_20_02_2024.yaml',
           patience=5,
           iterations=300,
           plots=False, 
           save=False, 
           val=False,
           batch=32)

## Generate YOLOv8 Models

In [2]:
# MODEL TRAINING
# model = YOLO('yolov8n.yaml')
# model = YOLO('yolov8n.pt')

model = YOLO('yolov8s-p2.yaml').load('yolov8s.pt')

results = model.train(data='crystals_20_02_2024.yaml', 
                      epochs=100, 
                      patience=10, 
                      device=0, 
                      pretrained=False,
                      batch=32)

Transferred 219/437 items from pretrained weights
New https://pypi.org/project/ultralytics/8.1.16 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.11 🚀 Python-3.11.8 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8s-p2.yaml, data=crystals_20_02_2024.yaml, epochs=100, time=None, patience=10, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train11, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False

train: Scanning C:\GitHub\mt2024-YOLOv8-ensemble\datasets\crystals_20_02_2024\labels\train.cache... 1040 images, 7 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<?, ?it/s]
val: Scanning C:\GitHub\mt2024-YOLOv8-ensemble\datasets\crystals_20_02_2024\labels\val.cache... 260 images, 4 backgrounds, 0 corrupt: 100%|██████████| 260/260 [00:00<?, ?it/s]


Plotting labels to c:\GitHub\mt2024-YOLOv8-ensemble\runs\detect\train11\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 70 weight(decay=0.0), 79 weight(decay=0.0005), 78 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to c:\GitHub\mt2024-YOLOv8-ensemble\runs\detect\train11
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      13.3G      3.141      4.592      4.152         79        640: 100%|██████████| 33/33 [01:15<00:00,  2.29s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:18<00:00,  3.61s/it]

                   all        260        958    0.00185      0.253     0.0423    0.00836


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      8/100      13.9G      1.798      2.418      2.184         75        640: 100%|██████████| 33/33 [00:44<00:00,  1.35s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:18<00:00,  3.61s/it]

                   all        260        958      0.317      0.384      0.278      0.159



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      9/100      12.5G       1.74      2.276      2.108        128        640:  42%|████▏     | 14/33 [00:12<00:14,  1.28it/s]

## Evaluate
- P (Precision): The accuracy of the detected objects, indicating how many detections were correct.
- R (Recall): The ability of the model to identify all instances of objects in the images.


### Load Model

In [ ]:
model = YOLO('crystals1\\train\weights\\best.pt')

### Predict

In [ ]:
images = (['CVAT\images\\folderID570_plateID1570_batchID6358_wellNum85_profileID1_d1_r102723_ef.jpg']) 

# Perform prediction
results = model.predict(images, save = True)

for result in results:
    boxes = result.boxes

    conf = boxes.conf
    cls = boxes.cls

    # print the class probabilitiy and the class 
    print(conf.item())
    print(cls.item())


## Ensemble Models
Ensemble output is average of the probabilities.

### Load Models

In [ ]:
model1 = YOLO('crystals1\\train\weights\\best.pt')
model2 = YOLO('crystals2\\train\weights\\best.pt')
# model3 = YOLO('crystals3\\train\weights\\best.pt')
# model4 = YOLO('crystals4\\train\weights\\best.pt')
# model5 = YOLO('crystals5\\train\weights\\best.pt')

### Predict

In [ ]:
images = (['CVAT\images\\05ku_B6_ImagerDefaults_8.jpg']) 

results1 = model1.predict(images)
# results2 = model2.predict(images)

print(len(results1[0].boxes.conf))

# if len(results1[0].boxes.conf) == 0:
#     test = 0
# else:
#     test = results1[0].boxes.conf.item()

# print(test)


### Predict